So far, I was removing the datetime column. What is going to happen if I keep it? Also, the atemp can be restored as we no longer use linear regression. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

RANDOM_STATE = 31415

In [ ]:
# metric to optimize
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

scorer = make_scorer(lambda y_test, predictions: np.sqrt(mean_squared_error(y_test, predictions)))

In [ ]:
training_set = pd.read_csv('../input/train.csv')
training_set['datetime'] = training_set['datetime'].apply(lambda x: pd.to_datetime(x).timestamp())

In [ ]:
training_set.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Basic preprocessing which applies to all regression techniques (dependent variable: casual)
data = training_set.drop(columns = ['registered', 'count'])

X_train, X_test, y_train, y_test = train_test_split(data, data.casual, test_size=0.2, random_state = RANDOM_STATE)
X_train = X_train.drop(columns = ['casual'])
X_test = X_test.drop(columns = ['casual'])

Random forests are going to only compare values, so one code encoding or normalization does not do any difference for them. With one small exception. I have the weather encoded as a number, the bigger the number the worse the weather. It is not strictly categorical value because the order of them has a meaning. I am going to keep it as a number, we may end up with some interesting results.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
?np.random.uniform

In [ ]:
N_ITER = 50
model_params_grid = {
    'n_estimators': np.random.uniform(20, 200, N_ITER).astype(int)
}

casual_model = RandomizedSearchCV(RandomForestRegressor(), model_params_grid, n_iter = N_ITER, n_jobs = -1, random_state = RANDOM_STATE, cv = 20)

In [ ]:
casual_model.fit(X_train, y_train)
casual_model = casual_model.best_estimator_

In [ ]:
# Same thing for the second variable
# Basic preprocessing which applies to all regression techniques (dependent variable: casual)
data = training_set.drop(columns = ['casual', 'count'])

X_train, X_test, y_train, y_test = train_test_split(data, data.registered, test_size=0.2, random_state = RANDOM_STATE)
X_train = X_train.drop(columns = ['registered'])
X_test = X_test.drop(columns = ['registered'])

In [ ]:
from sklearn.model_selection import cross_val_score

registered_model = RandomizedSearchCV(RandomForestRegressor(), model_params_grid, n_iter = N_ITER, n_jobs = -1, random_state = RANDOM_STATE, cv = 20)

In [ ]:
registered_model.fit(X_train, y_train)
registered_model = registered_model.best_estimator_

In [ ]:
# Final prediction of the baseline models, as I am not going to tweak them, I will move directly to the test data

test_dataset = pd.read_csv("../input/test.csv")
dates = test_dataset['datetime']
test_dataset['datetime'] = test_dataset['datetime'].apply(lambda x: pd.to_datetime(x).timestamp())

In [ ]:
test_data = test_dataset

In [ ]:
casual = casual_model.predict(test_data)
registered = registered_model.predict(test_data)
total = casual + registered

In [ ]:
test_dataset['datetime'] = dates
test_dataset['count'] = pd.Series(total)

In [ ]:
test_dataset[test_dataset['count'] < 0]

In [ ]:
test_dataset.loc[test_dataset['count'] < 0, 'count'] = 0

In [ ]:
test_dataset[test_dataset['count'] <= 0]

In [ ]:
test_dataset[['datetime', 'count']].to_csv('result.csv', index = False)